1:https://sourceforge.net/projects/tesseract-ocr.mirror/


2:https://github.com/oschwartz10612/poppler-windows/releases

In [ ]:
import pytesseract
import numpy as np
import cv2
import re
import json
from PIL import Image
from typing import Dict
TESSERACT_ENGINE_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
pytesseract.pytesseract.tesseract_cmd = TESSERACT_ENGINE_PATH
IMAGE_PATH = r"C:\Users\kgopi\Downloads\model\img5.png"
def preprocess_image(img: Image.Image) -> np.ndarray:
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(resized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 63, 12)
    return processed_image
def get_field(field_name: str, text: str) -> str:
    pattern_dict = {
        "patient_name": {"pattern": r"Name:\s*([A-Za-z\s]+)", "flags": 0},
        "patient_address": {"pattern": r"Address:\s*([A-Za-z0-9\s,]+)", "flags": 0},
        "medicines": {"pattern": r"Directions:\s*([\s\S]*?)Refill", "flags": re.DOTALL},
        "directions": {"pattern": r"Directions:\s*(.*)", "flags": re.DOTALL},
        "refill": {"pattern": r"Refill:\s*(\d+)", "flags": 0}
    }
    pattern_object = pattern_dict.get(field_name)
    if pattern_object:
        matches = re.findall(pattern_object["pattern"], text, flags=pattern_object["flags"])
        if matches:
            return matches[0].replace("\n", " ").strip()
    return None
def parse(text: str) -> Dict:
    return {
        "patient_name": get_field("patient_name", text),
        "patient_address": get_field("patient_address", text),
        "medicines": get_field("medicines", text),
        "directions": get_field("directions", text),
        "refill": get_field("refill", text)
    }
image = Image.open(IMAGE_PATH)
img = preprocess_image(image)
text = pytesseract.image_to_string(img, lang="eng").replace("\n", " ")
info = parse(text)
json_output = json.dumps(info, indent=4)
print(json_output)